In [31]:
# is base model we have here good enough?
# explore improvements
  # Maybe different flavor of bert? 
  # Maybe try a few different models and measure F1 scores?
  # Maybe try and fine tune on a different dataset?
  # chunk in a way that we tend to get full sentances? Right now a chunk can stop at a random spot in the sentance
    
  # a few ideas that can help with first three improvemetns are here https://towardsdatascience.com/which-flavor-of-bert-should-you-use-for-your-qa-task-6d6a0897fb24

In [1]:
# script to fine tune the model (it's fine tuned on squad 2.0)

# !python run_squad.py  \
#     --model_type bert   \
#     --model_name_or_path bert-base-uncased  \
#     --output_dir models/bert/ \
#     --data_dir dataset/   \
#     --overwrite_output_dir \
#     --overwrite_cache \
#     --do_train  \
#     --train_file train-v2.0.json   \
#     --version_2_with_negative \
#     --do_lower_case  \
#     --do_eval   \
#     --predict_file dev-v2.0.json   \
#     --per_gpu_train_batch_size 2   \
#     --learning_rate 3e-5   \
#     --num_train_epochs 2.0   \
#     --max_seq_length 384   \
#     --doc_stride 128   \
#     --threads 10   \
#     --save_steps 5000 

In [2]:
import torch
import wikipedia
import tensorflow as tf
import numpy as np

from collections import OrderedDict
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TFAutoModelForQuestionAnswering

In [3]:
tokenizer = AutoTokenizer.from_pretrained("models/bert")
model = TFAutoModelForQuestionAnswering.from_pretrained("models/bert", from_pt=True)

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

All the weights of TFBertForQuestionAnswering were initialized from the TF 2.0 model.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [64]:
question = "Who was the first president of the United States?"

q_search = wikipedia.search(question)
page = wikipedia.page(q_search[0])
context = page.content

In [65]:
inputs = tokenizer(question, context, add_special_tokens=True, return_tensors="tf")

Token indices sequence length is longer than the specified maximum sequence length for this model (10 > 512). Running this sequence through the model will result in indexing errors


In [80]:
inputs

{'input_ids': <tf.Tensor: shape=(1, 769), dtype=int32, numpy=
array([[  101,  2040,  2001,  1996,  2034,  2343,  1997,  1996,  2142,
         2163,  1029,   102,  1996,  2343,  1997,  1996,  2142,  2163,
         2003,  1996,  2132,  1997,  2110,  1998,  2132,  1997,  2231,
         1997,  1996,  2142,  2163,  1010, 17351,  2700,  2000,  1037,
         1018,  1011,  2095,  2744,  2011,  1996,  2111,  2083,  1996,
         6092,  2267,  1012,  1996,  2436, 14528,  5260,  1996,  3237,
         3589,  1997,  1996,  2976,  2231,  1998,  2003,  1996,  3474,
         1011,  1999,  1011,  2708,  1997,  1996,  2142,  2163,  4273,
         2749,  1012,  2144,  1996,  2436,  2001,  2511,  1999, 13739,
         1010,  4008,  2273,  2031,  2366,  2004,  2343,  1012,  1996,
         2034,  1010,  2577,  2899,  1010,  2180,  1037, 13604,  3789,
         1997,  1996,  6092,  2267,  1012, 25242,  6044,  2366,  2048,
         2512,  1011,  5486,  3408,  1999,  2436,  1006,  1996,  2069,
         2343, 

In [67]:
# answering wikipedia questions

In [81]:
max_length = model.config.max_position_embeddings

In [82]:
max_length

512

In [83]:
question_mask = inputs["token_type_ids"] == 0

In [95]:
question_tensor = tf.boolean_mask(inputs["input_ids"], question_mask)

In [96]:
question_tensor

<tf.Tensor: shape=(12,), dtype=int32, numpy=
array([ 101, 2040, 2001, 1996, 2034, 2343, 1997, 1996, 2142, 2163, 1029,
        102], dtype=int32)>

In [89]:
chunk_size = max_length - len(question_tensor) - 1

In [90]:
chunked_input = OrderedDict()

In [91]:
def get_chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [101]:
for k, v in inputs.items():
    question = tf.boolean_mask(v, question_mask)
    context =  tf.boolean_mask(v, ~question_mask)
    chunks = list(get_chunks(context, chunk_size))
    
    for i, chunk in enumerate(chunks):
        if i not in chunked_input:
            chunked_input[i] = {}
            
        entire_input = tf.concat([question, chunk], axis=0)
        
        if i != len(chunks) - 1:
            if k == 'input_ids':
                entire_input = tf.concat([entire_input, tf.constant([102])], axis=0)
            else:
                entire_input = tf.concat([entire_input, tf.constant([1])], axis=0)
        
        chunked_input[i][k] = tf.reshape(entire_input, [1, entire_input.shape[0]])

In [102]:
chunked_input[0]

{'input_ids': <tf.Tensor: shape=(1, 512), dtype=int32, numpy=
 array([[  101,  2040,  2001,  1996,  2034,  2343,  1997,  1996,  2142,
          2163,  1029,   102,  1996,  2343,  1997,  1996,  2142,  2163,
          2003,  1996,  2132,  1997,  2110,  1998,  2132,  1997,  2231,
          1997,  1996,  2142,  2163,  1010, 17351,  2700,  2000,  1037,
          1018,  1011,  2095,  2744,  2011,  1996,  2111,  2083,  1996,
          6092,  2267,  1012,  1996,  2436, 14528,  5260,  1996,  3237,
          3589,  1997,  1996,  2976,  2231,  1998,  2003,  1996,  3474,
          1011,  1999,  1011,  2708,  1997,  1996,  2142,  2163,  4273,
          2749,  1012,  2144,  1996,  2436,  2001,  2511,  1999, 13739,
          1010,  4008,  2273,  2031,  2366,  2004,  2343,  1012,  1996,
          2034,  1010,  2577,  2899,  1010,  2180,  1037, 13604,  3789,
          1997,  1996,  6092,  2267,  1012, 25242,  6044,  2366,  2048,
          2512,  1011,  5486,  3408,  1999,  2436,  1006,  1996,  2069,
  

In [115]:
answer = ''

for k, chunk in chunked_input.items():
    start_scores, end_scores = model(chunk)

    answer_start = tf.argmax(start_scores, axis=1).numpy()[0]
    answer_end = (tf.argmax(end_scores, axis=1) + 1).numpy()[0]
    answ = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(chunk['input_ids'][0][answer_start:answer_end]))
    
    if answ != '[CLS]':
        answer += answ + " / "

In [116]:
answer

'george washington / '